In [ ]:
#In this program, we calculate the largest water cluster on Earth in the presence of different levels of water
#Libraries
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np
import os
from tqdm import tqdm
from scipy.ndimage import measurements
import time
import sympy
import concurrent.futures

In [ ]:
#This function gives the different resolution of the earth map. we use it to extract the orginal resolution.
#the code has been written by
def Etopo(lon_area, lat_area, resolution):
    ### Input
    # resolution: resolution of topography for both of longitude and latitude [deg]
    # (Original resolution is 0.0167 deg)
    # lon_area and lat_area: the region of the map which you want like [100, 130], [20, 25]
    ###
    ### Output
    # Mesh type longitude, latitude, and topography data
    ###

    # Read NetCDF data
    data = Dataset("ETOPO1_Bed_c_gdal.grd", "r")
    # Get data
    lon_range = data.variables['x_range'][:]
    lat_range = data.variables['y_range'][:]
    topo_range = data.variables['z_range'][:]
    spacing = data.variables['spacing'][:]
    dimension = data.variables['dimension'][:]
    z = data.variables['z'][:]
    lon_num = dimension[0]
    lat_num = dimension[1]
  
    # Prepare array
    lon_input = np.zeros(lon_num); lat_input = np.zeros(lat_num)
    for i in range(lon_num):
        lon_input[i] = lon_range[0] + i * spacing[0]
    for i in range(lat_num):
        lat_input[i] = lat_range[0] + i * spacing[1]

    # Create 2D array
    lon, lat = np.meshgrid(lon_input, lat_input)

    # Convert 2D array from 1D array for z value
    topo = np.reshape(z, (lat_num, lon_num))
  
    # Skip the data for resolution
    if ((resolution < spacing[0]) | (resolution < spacing[1])):
        print('Set the highest resolution')
    else:
        skip = int(resolution/spacing[0])
        lon = lon[::skip,::skip]
        lat = lat[::skip,::skip]
        topo = topo[::skip,::skip]

    topo = topo[::-1]
  
    # Select the range of map
    range1 = np.where((lon>=lon_area[0]) & (lon<=lon_area[1]))
    lon = lon[range1]; lat = lat[range1]; topo = topo[range1]
    range2 = np.where((lat>=lat_area[0]) & (lat<=lat_area[1]))
    lon = lon[range2]; lat = lat[range2]; topo = topo[range2]
  
    # Convert 2D again
    lon_num = len(np.unique(lon))
    lat_num = len(np.unique(lat))
    lon = np.reshape(lon, (lat_num, lon_num))
    lat = np.reshape(lat, (lat_num, lon_num))
    topo = np.reshape(topo, (lat_num, lon_num))

    return lon, lat, topo

In [ ]:
# convert degrees to radians
def degree2radians(degree):
    return degree*np.pi/180

In [ ]:
#defining the orginal resoluton of the data
resolution = 0.0167
# resolution = 1

lon_area = [-180., 180.]
lat_area = [-90., 90.]
# Get mesh-shape topography data
lon_topo, lat_topo, topo = Etopo(lon_area, lat_area, resolution)
lon_topo = degree2radians(lon_topo)
lat_topo = degree2radians(lat_topo)

In [ ]:
#The Course graining part. we define the of the course graining window with N and course grain it with thw function rebin
N = 10
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

Width = int(10800/N)
Length = int(21600/N)
topo = rebin(topo,(Width, Length))
lat_topo = rebin(lat_topo,(Width, Length))
lon_topo = rebin(lon_topo,(Width, Length))

In [ ]:
#we extract all the elevations and sort then in an ascending order
indexes = np.argsort(topo.flatten())
hieght = np.sort(topo.flatten())
#defining the parallelization variables
l = len(hieght)
d = sympy.divisors(l)
num_iter = d[int(len(d)/2)]
num_proc = d[int(len(d)/2) - 1]

In [ ]:
#water clusters

total_landmass = np.zeros(len(hieght))
big_cluster = np.zeros(len(hieght))
total_lat = np.sum(np.abs(np.cos(lat_topo)))
all_pos = np.shape(lat_topo)[0]*np.shape(lat_topo)[1]


squares1 = np.zeros((num_proc, num_iter))
squares2 = np.zeros((num_proc, num_iter))


def water_function(ii):
    total_landmass = np.zeros(num_iter)
    big_cluster = np.zeros(num_iter)
    iso_hight = np.zeros(num_iter)
    iterr = 0
    for i in range(ii*num_iter, ii*num_iter + num_iter):
        sphere = 0
        new_topo=np.zeros(hieght.shape, bool)
        new_topo[indexes[:(i+1)]] = True
        new_topo = new_topo.reshape((Width, Length))
        pp = (i+1)/(all_pos)
        lw, num = measurements.label(new_topo)
        for l in range(len(lw)):
            if(lw[l][0] != 0 and lw[l][-1] != 0 and lw[l][-1] != lw[l][0]):
                lw[lw == lw[l][0]] = lw[l][-1]

        biggest_size = 0
        elements = np.unique(lw)
        elements = elements[elements != 0]
        unique_clustersize = np.zeros(len(elements))
        iterat = 0
        for j in elements:
            unique_clustersize[iterat] = np.sum(lw == j)
            iterat +=1

        elements = elements[unique_clustersize.argsort()]

        a = 0
        if len(elements)>2:
            for n in elements[[-1,-2,-3]]:
                mask = (lw == n)
                a = np.sum(np.abs(np.cos((mask*lat_topo)[(mask*lat_topo) != 0])))
                if a>biggest_size:
                    biggest_size = a

        elif len(elements) == 2:
            for n in elements[[-1,-2]]:
                mask = (lw == n)
                a = np.sum(np.abs(np.cos((mask*lat_topo)[(mask*lat_topo) != 0])))
                if a>biggest_size:
                    biggest_size = a

        else:
            mask = (lw == elements[0])
            biggest_size = np.sum(np.abs(np.cos((mask*lat_topo)[(mask*lat_topo) != 0])))



        total_landmass[iterr] = pp
        big_cluster[iterr] = float(biggest_size / total_lat)
        iterr += 1    
    
    
        
    return ii, big_cluster, total_landmass, iso_hight

with concurrent.futures.ProcessPoolExecutor() as executor: 
    for row, result1, result2 in executor.map(water_function, range(num_proc)):
        print(row)
        squares1[row] = result1
        squares2[row] = result2


np.save("/home/complex/c++/Earth/big_cluster_" + str(N) + ".npy", squares1.flatten())
np.save("/home/complex/c++/Earth/total_landmass_" + str(N) + ".npy", squares2.flatten())